In [13]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


### Extract data from syarah.com

#### this code will collect all URL cars for this website

In [1]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from urllib3.exceptions import InsecureRequestWarning

def scroll_down_all(driver, pause_sec=2):
    """
    This function scrolls down the webpage until the end of the page is reached.
    
    Parameters:
    - driver: The web driver object used to interact with the browser.
    - pause_sec: The number of seconds to pause between each scroll. Default is 1 second.
    """

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_sec)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break

        last_height = new_height

# Disable insecure request warning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

url = 'https://syarah.com/en/cars/?has_discount=0'

options = FirefoxOptions()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

driver.get(url)
scroll_down_all(driver, pause_sec=2)
page_source = driver.page_source
driver.quit()

soup = BeautifulSoup(page_source, 'lxml')
cars = soup.find_all('a', {'class': 'SearchCard-module__card'})

import requests
from bs4 import BeautifulSoup

data = []  # Store car information
attributeError = []  # Store URLs of cars that have different price tags
num_of_car = 0

for car in cars:
    num_of_car += 1
    car_url = f"https://syarah.com{car['href']}"
    car_response = requests.get(car_url, verify=False)
    src = car_response.content
    car_soup = BeautifulSoup(src, 'lxml')
    div_tags = car_soup.find_all('div', {'class': 'carDetailsSection-module__CCContainer'})
    labelTags = car_soup.find_all('label')
    
    # Initialize variables
    brand = ''
    extension = ''
    color = ''
    fuel = ''
    gear = ''
    new_used = ''
    mileage = ''
    engine_size = ''
    price = ''
    year = ''
    model = ''

    for label in labelTags:
        label_text = label.get_text()
        parent_tag = label.parent

        if label_text == 'Brand:':
            brand = parent_tag.find('strong').text.strip()
        elif label_text == 'Extension:':
            extension = parent_tag.find('strong').text.strip()
        elif label_text == 'Exterior color:':
            color = parent_tag.find('strong').text.strip()
        elif label_text == 'Fuel:':
            fuel = parent_tag.find('strong').text.strip()
        elif label_text == 'Gear:':
            gear = parent_tag.find('strong').text.strip()
        elif label_text == 'Condition:':
            new_used = parent_tag.find('strong').text.strip()
            if new_used == 'used':
                mileage_div = div_tags[-1]
                mileageKM = mileage_div.find('strong').text.strip()
                mileage = mileageKM[:-3]
            else:
                mileage = '0'
        elif label_text == 'Engine Size:':
            engine_size = parent_tag.find('strong').text.strip()
            

    try:
        # Attempt to find the price tag using class 'PaymentTabBox-module__price'
        price_tag = car_soup.find_all('strong', class_='PaymentTabBox-module__price')
        priceSAR = price_tag[-1].text.strip()
        price = priceSAR[:-3]
    except IndexError:
        try:
            # If the first attempt fails, try to find the price tag using class 'CashPayment-module__priceCashON CashPayment-module__oldPrice'
            priceSAR = car_soup.find('strong', class_='CashPayment-module__priceCashON CashPayment-module__oldPrice').text.strip()
            price = priceSAR[:-3]
        except AttributeError as s:
            try:
                # If the second attempt fails, try to find the price tag using class 'CashPayment-module__priceContSub'
                priceSAR_tag = car_soup.find('div', class_='CashPayment-module__priceContSub')
                priceSAR = priceSAR_tag.find('strong').text.strip()
                price = priceSAR[:-3]
            except AttributeError as s:
                try:
                    # If the third attempt fails, try to find the price tag using class 'CashPayment-module__priceCashON'
                    priceSAR = car_soup.find('strong', class_='CashPayment-module__priceCashON').text.strip()
                    price = priceSAR[:-3]
                except AttributeError as s:
                    print("AttributeError occurred: ", s)
                    attributeError.append(car_url)

    try:    
        model_div = div_tags[1]
        model = model_div.find('strong').text.strip()

        year_div = div_tags[2]
        year = year_div.find('strong').text.strip()
    except IndexError:
        print('Empty Data!')
        print(car_url)
  
    data.append([brand, model, year, extension, color, fuel, gear, engine_size, mileage, price, new_used])
    print(brand, model, year, extension, color, fuel, gear, engine_size, mileage, price, new_used)

print('number of cars',num_of_car)

Chevrolet Captiva 2023 LS Black Solar Automatic 1.5 0 72,335 new
Nissan Sunny 2020 S الشكل الجديد Silver Solar CVT 1.6 115058 44,100 used
Hyundai Santafe 2023 Smart Silver Solar Automatic 2.5 0 121,900 new
Toyota Yaris 2021 Y White Solar Automatic 1.5 59461 57,000 used
Toyota Yaris 2023 Y Silver Solar CVT 1.3 0 68,425 new
Chevrolet Groove 2023 premier Silver Solar CVT 1.5 0 74,405 new
Dodge Charger 2019 SXT Burgundy Solar Automatic 3.6 155632 83,000 used
MG RX 2022 LUX Black Solar Automatic 2.0 0 119,600 new
Great Wall Poer 2022 Elite Des 2W White Diesel Normal Gear 2.0 0 65,366 new
Toyota Camry 2019 LE White Hybrid Automatic 2.5 47194 85,500 used
Toyota Hilux 2018 GL2 White Solar Normal Gear 2.7 165147 77,000 used
Changan UNI K 2023 Elite Grey Solar Automatic 2.0 0 144,930 new
Kia Cerato 2017 EX White Gasoline Automatic 1.6 138814 48,000 used
HAVAL Jolion 2022 Premium Red Solar Automatic 1.5 0 73,888 new
Kia Sportage 2018 LX White Solar Automatic 2.0 65167 65,500 used
Toyota Raize 202

Kia Pegas 2020 LX White Solar Automatic 1.4 122837 35,900 used
Kia Pegas 2021 LX leaden Solar Automatic 1.4 106570 35,900 used
Mitsubishi Attrage 2020 LL Grey Solar Automatic 1.2 63641 35,900 used
Kia Pegas 2020 LX White Solar Automatic 1.4 117267 35,900 used
Nissan Sunny 2020 SV leaden Solar Automatic 1.5 119264 35,900 used
Kia Rio 2020 LX White Solar Automatic 1.4 87665 35,900 used
Kia Rio 2020 LX Silver Solar Automatic 1.4 101345 35,900 used
Kia Rio 2020 LX Silver Solar Automatic 1.4 105680 35,900 used
Kia Rio 2021 LX Silver Solar Automatic 1.4 99774 35,900 used
Kia Rio 2021 LX White Solar Automatic 1.4 118085 35,900 used
Kia Pegas 2020 LX White Solar Automatic 1.4 121823 36,000 used
Kia Pegas 2021 LX Red Solar Automatic 1.4 101244 36,000 used
Kia Rio 2020 LX White Solar Automatic 1.4 108392 36,000 used
Kia Rio 2020 LX Silver Solar Automatic 1.4 107568 36,000 used
Nissan Sunny 2019 SV Silver Solar Automatic 1.5 154047 36,100 used
Kia Pegas 2021 LX White Solar Automatic 1.4 117491 36

Hyundai Accent 2020 GL White Solar Automatic 1.4 158011 40,900 used
Kia Rio 2021 LX White Solar Automatic 1.4 65976 41,000 used
Kia Rio 2021 LX Grey Solar Automatic 1.4 101448 41,000 used
Hyundai Accent 2020 GL White Solar Automatic 1.4 132647 41,000 used
Hyundai Accent 2020 GL Grey Solar Automatic 1.4 160070 41,000 used
Kia Rio 2020 LX Silver Solar Automatic 1.4 88807 41,100 used
Hyundai Accent 2020 GL Silver Solar Automatic 1.4 101569 41,100 used
Hyundai Accent 2020 GL Silver Solar Automatic 1.4 140254 41,100 used
Chery Arrizo 5 2022 Luxury White Solar CVT 1.5 32564 41,200 used
Kia Pegas 2021 LX White Solar Automatic 1.4 107968 41,300 used
Nissan Sunny 2020 SV الشكل الجديد Silver Solar Automatic 1.6 103108 41,300 used
Chery Arrizo 5 2022 Luxury Red Solar Automatic 1.5 37499 41,400 used
Hyundai Accent 2020 GL Silver Solar Automatic 1.4 115154 41,400 used
Honda City 2020 DX White Solar Automatic 1.5 115999 41,400 used
Nissan Sunny 2020 S الشكل الجديد Silver Solar Automatic 1.6 71920 41

Hyundai Accent 2020 GL Silver Solar Automatic 1.4 113086 45,100 used
Kia Rio 2021 LX White Solar Automatic 1.4 119355 45,100 used
Toyota Yaris 2019 Y White Solar Automatic 1.5 179038 45,100 used
Hyundai Accent 2020 GL White Solar Automatic 1.4 87705 45,100 used
Toyota Yaris 2019 Y Silver Solar Automatic 1.5 135204 45,100 used
Hyundai Accent 2020 GL White Solar Automatic 1.4 93317 45,100 used
Hyundai Accent 2020 GL Silver Solar Automatic 1.4 84793 45,200 used
Toyota Yaris 2019 Y Silver Solar CVT 1.5 147362 45,200 used
Kia Rio 2021 LX Silver Solar Automatic 1.4 114413 45,300 used
Nissan Sunny 2021 SV White Solar Automatic 1.5 95061 45,300 used
Nissan Sunny 2021 Classic Brown Solar Automatic 1.5 52837 45,300 used
Hyundai Accent 2020 GL Grey Solar Automatic 1.4 135691 45,400 used
Kia Rio 2021 LX Silver Solar Automatic 1.4 43181 45,400 used
Hyundai Accent 2020 GL White Solar Automatic 1.4 98160 45,400 used
Hyundai Elantra 2018 GL White Solar Automatic 1.6 126039 45,400 used
Nissan Sunny 202

Hyundai Accent 2021 Smart White Solar Automatic 1.4 108497 50,100 used
Hyundai Accent 2021 Smart White Solar Automatic 1.4 62063 50,300 used
Hyundai Elantra 2020 GL Silver Solar Automatic 1.6 117629 50,300 used
Toyota Yaris 2021 Y Silver Solar Automatic 1.5 61315 50,500 used
Hyundai Accent 2021 Smart Grey Solar Automatic 1.4 106628 50,600 used
Hyundai Accent 2021 Smart White Solar Automatic 1.4 93237 50,600 used
Hyundai Accent 2021 Smart Silver Solar Automatic 1.4 69873 50,600 used
Hyundai Accent 2021 Smart White Solar Automatic 1.4 93814 50,600 used
Hyundai Accent 2021 Smart White Solar Automatic 1.4 151677 50,600 used
Toyota Yaris 2021 Y Silver Solar Automatic 1.5 79846 50,600 used
Hyundai Elantra 2019 GL White Solar Automatic 1.6 152979 50,700 used
Hyundai Accent 2021 Smart White Solar Automatic 1.4 81125 50,700 used
Hyundai Accent 2021 Smart White Solar Automatic 1.4 60713 50,700 used
Renault Megane 2021 PE Silver Solar CVT 1.6 22694 50,700 used
Hyundai Accent 2021 Smart Grey Solar

...

BMW 3 Series 2021 320i White Gasoline Automatic 2.0 32124 166,000 used
Lexus ES 2019 350 Silver Gasoline Automatic 3.5 90458 166,000 used
Jaguar XE 2020  Grey Solar Automatic 2.0 44500 176,500 used
Toyota FJ Cruiser 2022 FJ1 White Solar Automatic 4.0 37682 166,000 used
Lexus LS 2015 460L Grey Gasoline Automatic 4.6 195953 166,000 used
Chrysler S300 2022 Full White Solar Automatic 3.6 17088 166,500 used
Lexus ES 2020 250 AA Elegant Silver Gasoline Automatic 2.5 56700 166,850 used
Ford Expedition 2019 XLT Petrol Gasoline Automatic 3.5 114408 167,000 used
Chevrolet Tahoe 2019 LS Black Gasoline Automatic 5.3 69411 167,000 used
Toyota LAND CRUISER 70 2023 HARDTOP 70th LX1 3 Doors White Gasoline Manual 4.0 0 167,785 new
Toyota Haice 2023 ركاب White Diesel Normal Gear 2.8 0 145,900 new
Nissan Patrol 2016 LE Platinum White Solar Automatic 5.6 63155 168,000 used
Toyota Highlander 2022 GLE White Hybrid Automatic 2.5 20069 168,000 used
Lexus ES 2020 250 AA Elegant Dark Blue Gasoline Automatic 2.5

Toyota Land Cruiser Pickup 2023 DSL DLX Beige Diesel Normal Gear 4.5 0 161,900 new
Toyota Hilux 2022 SGLX 4X4 AT DSL White Diesel Automatic 2.4 0 187,220 new
GMC Sierra 2022 SLE another_color Gasoline Automatic 5.3 0 187,450 new
Toyota Highlander 2023 GLE White Hybrid CVT 2.5 0 187,680 new
Toyota Avalon 2022 Limited White Gasoline Automatic 3.5 0 187,680 new
Toyota Hilux 2023 SGLX White Diesel Automatic 2.8 0 187,680 new
GMC Yukon 2021 SLE Black Gasoline Automatic 5.3 75198 188,000 used
GMC Yukon 2020 SLE Black Solar Automatic 5.3 30961 188,000 used
BMW 5 Series 2021 520i Nardo Gray Gasoline Automatic 2.0 34415 189,000 used
GMC Yukon 2019 XL SLT Dark Blue Gasoline Automatic 5.3 69567 189,000 used
Toyota Land Cruiser Pickup 2022 LX Beige Diesel Manual 4.5 0 189,175 new
Chevrolet Tahoe 2021 LS leaden Gasoline Automatic 5.3 60545 189,500 used
Lexus IS 2023 300 AA Elegant Grey Gasoline Automatic 2.0 0 189,635 new
Hyundai Palisade 2023 Comfort another_color Gasoline Automatic 3.8 0 189,750 

Nissan Patrol 2021 Platinum Silver Gasoline Automatic 4.0 107991 221,000 used
BMW 5 Series 2021 520i Black Gasoline Automatic 2.0 8960 221,000 used
Lexus ES 2020 350 FF F Sport Black Gasoline Automatic 3.5 25852 221,000 used
Hongqi H9 2021  Golden Gasoline Automatic 2.0 6500 221,000 used
Lexus RX 2021 350 BB Excellence Grey Solar Automatic 5.3 12455 221,000 used
Lexus ES 2022 300 AH Hybrid Grey Hybrid CVT 2.5 20200 222,000 used
Lexus ES 2023 350 CA Grey Gasoline Automatic 3.5 0 194,300 new
Toyota Land Cruiser 2019 GXR White Diesel Automatic 4.5 55354 223,500 used
Lexus NX 2023 350 AA Elegant Grey Gasoline Automatic 2.4 0 224,595 new
Lexus RX 2020 450 BH Hybrid Silver Hybrid CVT 3.5 121070 225,000 used
GMC Sierra 2022 AT4 Blue Gasoline Automatic 5.3 100 226,000 used
Lexus ES 2021 350 FF F Sport Blue Gasoline Automatic 3.5 56110 226,000 used
Toyota Prado 2023 VX White Solar CVT 4.0 0 226,400 new
GMC Sierra 2022 AT4 Beige Gasoline Automatic 5.3 0 227,930 new
Cadillac Escalade 2019 Platinu

Nissan Patrol 2022 Platinum Dark Blue Gasoline Automatic 4.0 0 270,940 new
Nissan Patrol 2023 Platinum Dark Blue Solar Automatic 4.0 5270 271,000 used
Range Rover Range Rover 2018 SE Black Gasoline Automatic 3.0 150639 271,000 used
Chevrolet Tahoe 2023 LT White Solar Automatic 5.3 8028 271,500 used
Mercedes GLA 2023 200 White Solar Automatic 1.3 100 273,000 used
Toyota Land Cruiser 2021 Grand Touring Black Gasoline Automatic 4.6 47171 273,000 used
Mercedes CLA 2022 250 Black Gasoline Automatic 2.0 6224 274,500 used
Mercedes S 2016 400 White Gasoline Automatic 3.0 18410 276,000 used
Chevrolet Suburban 2021 Premier Brown Solar Automatic 5.3 10373 276,000 used
Nissan Patrol 2022 Platinum Black Gasoline Automatic 4.0 0 276,805 new
Nissan Patrol 2022 Platinum Black Gasoline Automatic 4.0 0 276,805 new
Chevrolet Tahoe 2023 LT Black Gasoline Automatic  0 277,725 new
Toyota Land Cruiser 2021 Grand Touring White Gasoline Automatic 4.6 57517 278,000 used
Toyota Land Cruiser 2019 VXS White Gasoli

Ford F150 2022 Raptor Red Gasoline Automatic 3.5 2585 395,000 used
Lexus LX 2021 570 S Silver Gasoline Automatic 5.7 149301 401,000 used
Lexus LX 2021 570 S White Solar Automatic 5.7 78285 411,000 used
Dodge Ram 2022 TRX Silver Gasoline Automatic 6.2 11931 415,000 used
Audi A8 2021  Dark Blue Gasoline Automatic 3.0 5087 421,000 used
Lincoln Navigator 2022 بريزيدنشيل Black Gasoline Automatic 3.5 0 429,295 new
Mercedes S 2020 560 Black Gasoline Automatic 4.0 33855 432,000 used
BMW X Series 2023 6 xDrive 40i Brown Gasoline Automatic 3.0 12680 435,000 used
Toyota Land Cruiser 2023 VX White Solar Automatic 3.5 0 380,600 new
Lexus LX 2021 570 S Black Gasoline Automatic 5.7 43361 448,000 used
Audi A8 2022  White Gasoline Automatic 3.0 2850 450,500 used
Lexus LX 2021 570 S Blue Gasoline Automatic 5.7 37830 451,000 used
Lexus LX 2021 570 Black Ed. Black Gasoline Automatic 5.7 31657 456,000 used
Range Rover Range Rover 2022 Sport HSE Bronze Gasoline Automatic 3.0 12692 475,000 used
Toyota Land C

In [2]:
import pandas as pd 
import csv

In [3]:
df = pd.DataFrame(data, columns = ['brand', 'model', 'year', 'Extension', 'color', 'fuel', 'gear', 'engine_size', 'Mileage', 'price', 'state'])
df

,brand,model,year,Extension,color,fuel,gear,engine_size,Mileage,price,state
0,Chevrolet,Captiva,2023,LS,Black,Solar,Automatic,1.5,0,"72,335",new
1,Nissan,Sunny,2020,S الشكل الجديد,Silver,Solar,CVT,1.6,115058,"44,100",used
2,Hyundai,Santafe,2023,Smart,Silver,Solar,Automatic,2.5,0,"121,900",new
3,Toyota,Yaris,2021,Y,White,Solar,Automatic,1.5,59461,"57,000",used
4,Toyota,Yaris,2023,Y,Silver,Solar,CVT,1.3,0,"68,425",new
...,...,...,...,...,...,...,...,...,...,...,...
4495,Range Rover,Range Rover,2023,V8P,White,Solar,Automatic,4.4,7448,"921,000",used
4496,Range Rover,Range Rover,2023,HSE,White,Solar,Automatic,4.4,135,"956,000",used
4497,Rolls Royce,Dawn,2017,,leaden,Solar,Automatic,6.6,34088,"985,000",used
4498,Range Rover,Range Rover,2023,HSE,Black,Solar,Automatic,4.4,0,"861,700",new


In [4]:
df.to_csv("C:\\Users\\hp\\OneDrive\\Desktop\\used_car\\dataframeCars(webScrapping).csv")